**Задание 1.** Точность почти в 10 раз меньше необходимой, ошибка на этапе построения спрямляющего пространства (3/5 за старания и доведения до конца?)

**Задание 2.** Не проведена кросс-валидация для градиентного бустинга, не дождался окончания работы классического линейного SVM (0.5/3?)

**Задание 3.** Нет пункта (2), написал код, но не произвёл вычисления для двух других пунктов (0.2/2?)

**Штраф** -1 балл за отправку после мягкого дедлайна.

# Машинное обучение, ФКН ВШЭ

## Практическое задание 8. Метод опорных векторов и аппроксимация ядер

### Общая информация
Дата выдачи: 05.02.2021

Мягкий дедлайн: 01:59MSK 21.02.2021

Жесткий дедлайн: 01:59MSK 24.02.2021

### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимальная оценка за работу (без учёта бонусов) — 10 баллов.

Сдавать задание после указанного жёсткого срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.

### Формат сдачи
Задания сдаются через систему anytask. Посылка должна содержать:
* Ноутбук homework-practice-08-random-features-Username.ipynb

Username — ваша фамилия и имя на латинице именно в таком порядке

### О задании

На занятиях мы подробно обсуждали метод опорных векторов (SVM). В базовой версии в нём нет чего-то особенного — мы всего лишь используем специальную функцию потерь, которая не требует устремлять отступы к бесконечности; ей достаточно, чтобы отступы были не меньше +1. Затем мы узнали, что SVM можно переписать в двойственном виде, который, позволяет заменить скалярные произведения объектов на ядра. Это будет соответствовать построению модели в новом пространстве более высокой размерности, координаты которого представляют собой нелинейные модификации исходных признаков.

Ядровой SVM, к сожалению, довольно затратен по памяти (нужно хранить матрицу Грама размера $d \times d$) и по времени (нужно решать задачу условной оптимизации с квадратичной функцией, а это не очень быстро). Мы обсуждали, что есть способы посчитать новые признаки $\tilde \varphi(x)$ на основе исходных так, что скалярные произведения этих новых $\langle \tilde \varphi(x), \tilde \varphi(z) \rangle$ приближают ядро $K(x, z)$.

Мы будем исследовать аппроксимации методом Random Fourier Features (RFF, также в литературе встречается название Random Kitchen Sinks) для гауссовых ядер. Будем использовать формулы, которые немного отличаются от того, что было на лекциях (мы добавим сдвиги внутрь тригонометрических функций и будем использовать только косинусы, потому что с нужным сдвигом косинус превратится в синус):
$$\tilde \varphi(x) = (
\cos (w_1^T x + b_1),
\dots,
\cos (w_n^T x + b_n)
),$$
где $w_j \sim \mathcal{N}(0, 1/\sigma^2)$, $b_j \sim U[-\pi, \pi]$.

На новых признаках $\tilde \varphi(x)$ мы будем строить любую линейную модель.

Можно считать, что это некоторая новая парадигма построения сложных моделей. Можно направленно искать сложные нелинейные закономерности в данных с помощью градиентного бустинга или нейронных сетей, а можно просто нагенерировать большое количество случайных нелинейных признаков и надеяться, что быстрая и простая модель (то есть линейная) сможет показать на них хорошее качество. В этом задании мы изучим, насколько работоспособна такая идея.

### Алгоритм

Вам потребуется реализовать следующий алгоритм:
1. Понизить размерность выборки до new_dim с помощью метода главных компонент.
2. Для полученной выборки оценить гиперпараметр $\sigma^2$ с помощью эвристики (рекомендуем считать медиану не по всем парам объектов, а по случайному подмножеству из где-то миллиона пар объектов): $$\sigma^2 = \text{median}_{i, j = 1, \dots, \ell, i \neq j} \left\{\sum_{k = 1}^{d} (x_{ik} - x_{jk})^2 \right\}$$
3. Сгенерировать n_features наборов весов $w_j$ и сдвигов $b_j$.
4. Сформировать n_features новых признаков по формулам, приведённым выше.
5. Обучить линейную модель (логистическую регрессию или SVM) на новых признаках.
6. Повторить преобразования (PCA, формирование новых признаков) к тестовой выборке и применить модель.

Тестировать алгоритм мы будем на данных Fashion MNIST. Ниже код для их загрузки и подготовки.

In [1]:
import keras
from keras.datasets import fashion_mnist
(x_train_pics, y_train), (x_test_pics, y_test) = fashion_mnist.load_data()
x_train = x_train_pics.reshape(x_train_pics.shape[0], -1)
x_test = x_test_pics.reshape(x_test_pics.shape[0], -1)

Импортируем библиотеки для имплементации алгоритма

In [4]:
from sklearn.decomposition import PCA
import numpy as np
import random
#import itertools
from statistics import median
from math import pi
from math import cos

# Проверка алгоритма на верность вне класса

Сначала построим алгоритм вне класса, чтобы проверить его на верность выполнения (для простоты будем использовать n_components=2)

In [152]:
x_1 = x_train
pca_train = PCA(n_components=50).fit_transform(x_1)

In [153]:
already_used = [[]]
sigma = []
for k in range((10**6)*2):
  i = random.randint(0, len(pca_train)-1)
  j = random.randint(0, len(pca_train)-1)
  if i!=j and np.any(already_used != [i, j]) and np.any(already_used != [j, i]):
    #добавляем комбинацию в массив, чтобы гарантировать отсутствие повторений
    already_used.append([i, j])
    sigma_i=0
    # pca_train.shape[1] = n_components => считаем квадрат отклонения по всем признакам между двумя парами объектов
    for d in range(pca_train.shape[1]):
      sigma_i += (pca_train[i][d]-pca_train[j][d])**2
    sigma.append([sigma_i])

In [154]:
#Оцениваем гиперпараметр на полученной выборке
sigma = np.array(sigma)
sigma_median = median(sigma)

In [74]:
#Проверяем, что было сравнено действительно много пар объектов
len(sigma)

20000

In [155]:
w = np.random.normal(0, 1/sigma_median, 1000*50)
w = np.reshape(w, (1000, 50))

In [156]:
b = np.random.uniform(-pi, pi, 1000)

In [100]:
np.sum(np.reshape(w[1], (len(pca_train),1)) * pca_train, axis=1)

array([-3.86953494e-05, -2.69624385e-04, -3.14670900e-04, ...,
       -7.13280550e-06,  1.18547818e-05, -2.16234279e-04])

In [101]:
pca_train.shape

(60000, 20)

In [157]:
phi = []
for i in range(1000):
  phi.append([w[i] @ pca_train.T + b[i]])

phi = np.array(phi)

In [158]:
phi = np.vectorize(cos)(phi.reshape(len(pca_train),-1))

In [122]:
phi.shape

(60000, 100)

In [123]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
clf = make_pipeline(StandardScaler(), LinearSVC(random_state=0))
clf.fit(phi, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linearsvc',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='squared_hinge', max_iter=1000,
                           multi_class='ovr', penalty='l2', random_state=0,
                           tol=0.0001, verbose=0))],
         verbose=False)

Повторим манипуляции для тестовой выборки

In [159]:
pca_test = PCA(n_components=50).fit_transform(x_test)

In [161]:
already_used = [[]]
sigma = []
for k in range((10**6)*2):
  i = random.randint(0, len(pca_test)-1)
  j = random.randint(0, len(pca_test)-1)
  if i!=j and np.any(already_used != [i, j]) and np.any(already_used != [j, i]):
    #добавляем комбинацию в массив, чтобы гарантировать отсутствие повторений
    already_used.append([i, j])
    sigma_i=0
    # pca_test.shape[1] = n_components => считаем квадрат отклонения по всем признакам между двумя парами объектов
    for d in range(pca_test.shape[1]):
      sigma_i += (pca_test[i][d]-pca_test[j][d])**2
    sigma.append([sigma_i])

In [162]:
#Оцениваем гиперпараметр на полученной выборке
sigma = np.array(sigma)
sigma_median = median(sigma)

In [163]:
phi_test = []
for i in range(1000):
  phi_test.append([w[i] @ pca_test.T + b[i]])

phi_test = np.array(phi_test)

In [164]:
phi_test = np.vectorize(cos)(phi_test.reshape(len(pca_test),-1))

In [144]:
phi_test.shape

(10000, 100)

In [165]:
np.std(phi_test.T[1])

0.711397057083308

In [166]:
svm = LinearSVC(random_state=0)
svm.fit(phi, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
          verbose=0)

In [167]:
svm.score(phi_test, y_test)

0.0953

In [147]:
y_pred = clf.predict(phi_test)

In [146]:
clf.score(phi_test, y_test)

0.0984

In [94]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(random_state=0)

In [170]:
logreg.fit(phi, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [171]:
logreg.score(phi_test, y_test)

0.0983

In [176]:
phi_test[::1000]

array([[-0.68814859, -0.68803182, -0.68862717, ..., -0.68792254,
        -0.68847516, -0.68830349],
       [ 0.92044824,  0.92047345,  0.92055018, ...,  0.92039552,
         0.92044533,  0.92043013],
       [ 0.75954502,  0.75972819,  0.75969182, ...,  0.75977316,
         0.75948341,  0.75936504],
       ...,
       [-0.94531213, -0.94521868, -0.94515596, ..., -0.94512737,
        -0.94523074, -0.94522855],
       [ 0.57995871,  0.58022367,  0.5800974 , ...,  0.58016402,
         0.57993726,  0.57994272],
       [-0.21208242, -0.21188784, -0.21097547, ..., -0.21154633,
        -0.21176609, -0.21197081]])

In [177]:
logreg.predict(phi_test)[::1000]

array([8, 0, 0, 0, 0, 0, 8, 8, 0, 8], dtype=uint8)

Ощущение, что на этапе имплементации спрямляющего пространства я допустил грубую ошибку (сказалось ужасное знание линала) и поэтому модель где-то сломалась, выдавая лишь одну разделяющую прямую, превратившись в модель бинарной классификации, которая при этом в рамках двух классов выдаёт точность меньше константной (которая должна была быть 0.1)


# Выполнение задания

__Задание 1. (5 баллов)__

Реализуйте алгоритм, описанный выше. Можете воспользоваться шаблоном класса ниже или написать свой интерфейс.

Ваша реализация должна поддерживать следующие опции:
1. Возможность задавать значения гиперпараметров new_dim (по умолчанию 50) и n_features (по умолчанию 1000).
2. Возможность включать или выключать предварительное понижение размерности с помощью метода главных компонент.
3. Возможность выбирать тип линейной модели (логистическая регрессия или SVM с линейным ядром).

Протестируйте на данных Fashion MNIST, сформированных кодом выше. Если на тесте у вас получилась доля верных ответов не ниже 0.84 с гиперпараметрами по умолчанию, то вы всё сделали правильно.

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression


import numpy as np
import random
#import itertools
from statistics import median
from math import pi
from math import cos
class RFFPipeline(BaseEstimator, TransformerMixin):
    def __init__(self, n_features=1000, new_dim=50, use_PCA=True, classifier='logreg'):
        """        
        Implements pipeline, which consists of PCA decomposition,
        Random Fourier Features approximation and linear classification model.
        
        n_features, int: amount of synthetic random features generated with RFF approximation.

        new_dim, int: PCA output size.
        
        use_PCA, bool: whether to include PCA preprocessing.
        
        classifier, string: either 'svm' or 'logreg', a linear classification model to use on top of pipeline.
        
        Feel free to edit this template for your preferences.    
        """
        self.n_features = n_features
        self.use_PCA = use_PCA
        self.new_dim = new_dim
        self.classifier = classifier
        
    def fit(self, X, y):
        """
        Fit all parts of algorithm (PCA, RFF, Classification) to training set.
        """
        self.X = X
        self.y = y

        ## Этап понижения размерности
        if self.use_PCA==True:
          self.X = PCA(n_components=self.new_dim).fit_transform(self.X)
        ## Этап оценки гиперпараметра сигма
        already_used = [[]]
        sigma = []
        for k in range((10**6)*2):
          i = random.randint(0, len(self.X)-1)
          j = random.randint(0, len(self.X)-1)
          if i!=j and np.any(already_used != [i, j]) and np.any(already_used != [j, i]):
        #добавляем комбинацию в массив, чтобы гарантировать отсутствие повторений
            already_used.append([i, j])
            sigma_i=0
            # self.X.shape[1] = n_components => считаем квадрат отклонения по всем признакам между двумя парами объектов
            for d in range(self.X.shape[1]):
              sigma_i += (self.X[i][d]-self.X[j][d])**2
            sigma.append([sigma_i])
        #Оцениваем гиперпараметр на полученной выборке
        sigma = np.array(sigma)
        sigma_median = median(sigma)
        
        ## Генерируем случайные веса w, b
        w = np.random.normal(0, 1/sigma_median, self.n_features*self.new_dim)
        w = np.reshape(w, (self.n_features, self.new_dim))
        b = np.random.uniform(-pi, pi, self.n_features)
         
        ## Сформируем новые признаки
        phi = []
        for i in range(self.n_features):
          phi.append([w[i] @ self.X.T + b[i]])
        phi = np.array(phi)
        phi = np.vectorize(cos)(phi.reshape(len(self.X),-1))

        ## Обучаем выбранную модель
        if self.classifier=='logreg':
          self.clf = LogisticRegression(random_state=0)
        else:
          self.clf = LinearSVC(random_state=0)
        self.clf.fit(phi, self.y)

        # Your code here: (￣▽￣)/♫•*¨*•.¸¸♪
        return self

    def predict_proba(self, X):
        """
        Apply pipeline to obtain scores for input data.
        """
        
        # Your code here: (￣▽￣)/♫•*¨*•.¸¸♪
        raise NotImplementedError
        
    def predict(self, X):
        """
        Apply pipeline to obtain discrete predictions for input data.
        """
        self.X = X

        ## Этап понижения размерности
        if self.use_PCA==True:
          self.X = PCA(n_components=self.new_dim).fit_transform(self.X)
        ## Этап оценки гиперпараметра сигма
        already_used = [[]]
        sigma = []
        for k in range((10**6)*2):
          i = random.randint(0, len(self.X)-1)
          j = random.randint(0, len(self.X)-1)
          if i!=j and np.any(already_used != [i, j]) and np.any(already_used != [j, i]):
        #добавляем комбинацию в массив, чтобы гарантировать отсутствие повторений
            already_used.append([i, j])
            sigma_i=0
            # self.X.shape[1] = n_components => считаем квадрат отклонения по всем признакам между двумя парами объектов
            for d in range(self.X.shape[1]):
              sigma_i += (self.X[i][d]-self.X[j][d])**2
            sigma.append([sigma_i])
        #Оцениваем гиперпараметр на полученной выборке
        sigma = np.array(sigma)
        sigma_median = median(sigma)
        
        ## Генерируем случайные веса w, b
        w = np.random.normal(0, 1/sigma_median, self.n_features*self.new_dim)
        w = np.reshape(w, (self.n_features, self.new_dim))
        b = np.random.uniform(-pi, pi, self.n_features)
         
        ## Сформируем новые признаки
        phi = []
        for i in range(self.n_features):
          phi.append([w[i] @ self.X.T + b[i]])
        phi = np.array(phi)
        phi = np.vectorize(cos)(phi.reshape(len(self.X),-1))

        self.y_pred = self.clf.predict(phi)
        # Your code here: (￣▽￣)/♫•*¨*•.¸¸♪
        return self.y_pred

In [10]:
rff = RFFPipeline()
rff.fit(x_train, y_train)
y_pred = rff.predict(x_test)
accuracy_score(y_pred, y_test)

NotImplementedError: ignored

__Задание 2. (3 балла)__

Сравните подход со случайными признаками с обучением SVM на исходных признаках. Попробуйте вариант с обычным (линейным) SVM и с ядровым SVM. Ядровой SVM может очень долго обучаться, поэтому можно делать любые разумные вещи для ускорения: брать подмножество объектов из обучающей выборки, например.

Сравните подход со случайными признаками с вариантом, в котором вы понижаете размерность с помощью PCA и обучаете градиентный бустинг. Используйте одну из реализаций CatBoost/LightGBM/XGBoost, не забудьте подобрать число деревьев и длину шага.

Сделайте выводы — насколько идея со случайными признаками работает? Сравните как с точки зрения качества, так и с точки зрения скорости обучения и применения.

Попробуем обучить классический линейный классификатор SVM:

In [ ]:
classic_svm = make_pipeline(StandardScaler(), LinearSVC(random_state=0))
classic_svm.fit(x_train, y_train)

In [ ]:
classic_svm.score(x_test, y_test)

Попробуем использовать градиентный бустинг:



In [180]:
pca_train = PCA(n_components=50).fit_transform(x_train)
pca_test = PCA(n_components=50).fit_transform(x_test)

In [181]:
from sklearn.ensemble import GradientBoostingClassifier
gb_sklearn = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
                                        max_depth=1, random_state=0)
gb_sklearn.fit(pca_train, y_train)

NameError: ignored

In [183]:
gb_sklearn.score(pca_test, y_test)

0.4921

In [186]:
from catboost import CatBoostClassifier
gb_catboost = CatBoostClassifier(iterations=100)
gb_catboost.fit(pca_train, y_train)
gb_catboost.score(pca_test, y_test)

Learning rate set to 0.5
0:	learn: 1.2847812	total: 566ms	remaining: 56.1s
1:	learn: 1.0355214	total: 1.43s	remaining: 1m 10s
2:	learn: 0.9027130	total: 2.52s	remaining: 1m 21s
3:	learn: 0.8306724	total: 3.43s	remaining: 1m 22s
4:	learn: 0.7698570	total: 4.46s	remaining: 1m 24s
5:	learn: 0.7132667	total: 5.45s	remaining: 1m 25s
6:	learn: 0.6742825	total: 6.43s	remaining: 1m 25s
7:	learn: 0.6520250	total: 7.41s	remaining: 1m 25s
8:	learn: 0.6251048	total: 8.36s	remaining: 1m 24s
9:	learn: 0.5949967	total: 9.32s	remaining: 1m 23s
10:	learn: 0.5741014	total: 10.3s	remaining: 1m 23s
11:	learn: 0.5532143	total: 11.3s	remaining: 1m 23s
12:	learn: 0.5370275	total: 12.4s	remaining: 1m 22s
13:	learn: 0.5233742	total: 13.4s	remaining: 1m 22s
14:	learn: 0.5113189	total: 14.5s	remaining: 1m 22s
15:	learn: 0.5012152	total: 15.4s	remaining: 1m 20s
16:	learn: 0.4934987	total: 15.9s	remaining: 1m 17s
17:	learn: 0.4814401	total: 16.5s	remaining: 1m 15s
18:	learn: 0.4725334	total: 17s	remaining: 1m 12s


0.4917

Учитывая то, что оптимально реализованный RFF классификатор должен выдавать качество 0.84, а градиентный бустинг без кросс-валидации выдаёт 0.49, можем сделать вывод, что RFF позволяет на порядок увеличить качество предсказания классов и добиться неплохих результатов.

__Задание 3. (2 балла)__

Проведите эксперименты:
1. Помогает ли предварительное понижение размерности с помощью PCA? 
2. Как зависит итоговое качество от n_features? Выходит ли оно на плато при росте n_features?
3. Важно ли, какую модель обучать — логистическую регрессию или SVM?

1. Роль метода главных компонент в обучении

In [ ]:
%%time
rff = RFFPipeline(use_PCA=True)
rff.fit(x_train, y_train)
y_pred = rff.predict(x_test)
accuracy_score(y_pred, y_test)

In [ ]:
%%time
rff = RFFPipeline(use_PCA=False)
rff.fit(x_train, y_train)
y_pred = rff.predict(x_test)
accuracy_score(y_pred, y_test)

Сделаем вывод: как и предполагалось, понижение размерности ускоряет обучение, но при этом незначительно ухудшает качество модели.

Так, если первоочередной задачей для исследования является получение хоть какого-то правдоподобного результата как можно скорее, то следует понизить размерность, тогда как в случае попытки пробить наибольшую точность следует отказаться от понижения размерности

2. Зависимость между качеством и n_features

In [ ]:
#тут должны быть какие-нибудь красивые графики

3. Разница между логистической регрессией и SVM

Вообще говоря, очевидно, что при использовании SVM время обучения увеличивается в разы, что не очень приятно. С точки зрения качества, на моей поломанной модели оно оказывается примерно одинаковым, поэтому можно предположить, что разницы нет



In [ ]:
%%time
rff = RFFPipeline(classifier='logreg')
rff.fit(x_train, y_train)
y_pred = rff.predict(x_test)
accuracy_score(y_pred, y_test)

In [ ]:
%%time
rff = RFFPipeline(classifier='svm')
rff.fit(x_train, y_train)
y_pred = rff.predict(x_test)
accuracy_score(y_pred, y_test)

### Бонус

__Задание 4. (Максимум 2 балла)__

Как вы, должно быть, помните с курса МО-1, многие алгоритмы машинного обучения работают лучше, если признаки данных некоррелированы. Оказывается, что для RFF существует модификация, позволяющая получать ортогональные случайные признаки (Orthogonal Random Features, ORF). Об этом методе можно прочитать в [статье](https://proceedings.neurips.cc/paper/2016/file/53adaf494dc89ef7196d73636eb2451b-Paper.pdf). Реализуйте класс для вычисления ORF по аналогии с основным заданием. Обратите внимание, что ваш класс должен уметь работать со случаем n_features > new_dim (в статье есть замечание на этот счет). Проведите эксперименты, сравнивающие RFF и ORF, сделайте выводы.

In [ ]:
# Your code here: (￣▽￣)/♫•*¨*•.¸¸♪

__Задание 5. (Максимум 2 балла)__

Поэкспериментируйте с функциями для вычисления новых случайных признаков. Не обязательно использовать косинус от скалярного произведения — можно брать знак от него, хэш и т.д. Придумайте побольше вариантов для генерации признаков и проверьте, не получается ли с их помощью добиваться более высокого качества. Также можете попробовать другой классификатор поверх случайных признаков, сравните результаты.

In [ ]:
# Your code here: (￣▽￣)/♫•*¨*•.¸¸♪